In [ ]:
# The jupyter script is used to carry out the analysis of group runs
# this is an ETL process: extract (from database), transform (preprocessing techniques), load (numPy arrays)
# take average of music therpy group per task
# take average of control group per task
# run 12 second sliding window on each group per task
# dataset is resampled due to undersampling during the recording
# ICA, notch and bandpass filtering are implmeneted to mitigate eeg noise
# results: spectogram and average bandpower comparison betwen timepoints per channel per brainwave

In [ ]:
from helper import*
from params import*

In [ ]:
def avgBandPowerProcessingPipeline(**kwargs):
    # establish connection with music_therapy_eeg database
    db_connection = create_db_connection(hostName,userName,userPassword,databaseName)
    data = df_to_table_query(db_connection,query_1)
    # transform sql extracted table to raw eeg data
    rawEEG = (transformToRawEEG(data,fs,collection_time,fs_setting))[0]
    time_s = (transformToRawEEG(data,fs,collection_time,fs_setting))[1]
    # compute independent comonent analysis on raw eeg data
    ica_data = customICA(rawEEG,tuneval)
    # Filters: notch filter and bandpass filter
    noc=filters()
    notch_data = noc.notch(ica_data,line,fs,Q)
    bp = filters()
    bpData = bp.butterBandPass(notch_data,lowcut,highcut,fs,order)
    # run sliding window on bandpass filtered data
    rw_data = rollingWindow(bpData,window_size,step_size)
    # compute power spectrum of each window per channel
    delta_low = brainwaves['delta'][0]
    delta_high = brainwaves['delta'][1]
    theta_low = brainwaves['theta'][0]
    theta_high = brainwaves['theta'][1]
    alpha_low = brainwaves['alpha'][0]
    alpha_high = brainwaves['alpha'][1]
    beta_low = brainwaves['beta'][0]
    beta_high = brainwaves['beta'][1]
    gamma_low = brainwaves['gamma'][0]
    gamma_high = brainwaves['gamma'][1]
    # compute average band power for timepoint 1
    deltaAvgBandPower = averageBandPower(rw_data,'3D',fs,delta_low,delta_high,win)
    thetaAvgBandPower = averageBandPower(rw_data,'3D',fs,theta_low,theta_high,win)
    alphaAvgBandPower = averageBandPower(rw_data,'3D',fs,alpha_low,alpha_high,win)
    betaAvgBandPower = averageBandPower(rw_data,'3D',fs,beta_low,beta_high,win)
    gammaAvgBandPower = averageBandPower(rw_data,'3D',fs,gamma_low,gamma_high,win)
    return deltaAvgBandPower,thetaAvgBandPower,alphaAvgBandPower,betaAvgBandPower,gammaAvgBandPower